In [121]:
from lightgbm import LGBMRegressor, LGBMClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

# 読み込み

In [147]:
train_test = pd.read_csv('../input/yachida.csv')

# 4人のscoreの平均を取る
train_test = train_test.groupby(
    [col for col in train_test.columns if not col == 'score'], as_index=False)[['score']].mean()
train_test.head()

,eye_left_x,eye_left_y,eye_right_x,eye_right_y,nose_x,nose_y,mouse_x,mouse_y,score
0,0.003686,0.364129,0.412835,0.848756,0.285172,0.512789,0.912312,0.179207,2.50
1,0.007223,0.322489,0.063422,0.066738,0.120908,0.903612,0.590445,0.832553,2.50
2,0.013332,0.739455,0.381475,0.236677,0.812067,0.557608,0.592940,0.388272,2.00
3,0.019862,0.091261,0.043216,0.573151,0.069793,0.677142,0.183199,0.754341,1.25
4,0.022230,0.086808,0.630817,0.949252,0.651827,0.358567,0.196062,0.321173,1.25


In [148]:
train, test = train_test_split(train_test, test_size=0.1, shuffle=True, random_state=42)

In [149]:
folds = KFold(n_splits=3, shuffle=True, random_state=42)

oof_preds = np.zeros(train.shape[0])
sub = np.zeros(test.shape[0])
feature_importances = pd.DataFrame()
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train)):
    train_x = train.drop(columns='score').iloc[trn_idx]
    train_y = train['score'].iloc[trn_idx]
    valid_x = train.drop(columns='score').iloc[val_idx]
    valid_y = train['score'].iloc[val_idx]
    
    params = {
        'num_leaves': 10,
    }
    model = LGBMRegressor(n_estimators=10000,
                         metric='rmse',
                         learning_rate=0.001)
#     model = LGBMClassifier()
    model.fit(train_x, train_y,
             eval_set=[(train_x, train_y), (valid_x, valid_y)],
             early_stopping_rounds=500,
             eval_metric='rmse',
             verbose=1000)
    oof_preds[val_idx] = model.predict(valid_x)
    sub += model.predict(test.drop(columns='score')) / folds.n_splits
    fold_importance = pd.DataFrame()
    fold_importance['feature'] = train_x.columns
    fold_importance["importance"] = model.feature_importances_
    fold_importance["fold"] = n_fold + 1
    feature_importances = pd.concat(
        [feature_importances, fold_importance], axis=0)

Training until validation scores don't improve for 500 rounds.
[1000]	training's rmse: 0.550646	valid_1's rmse: 0.889482
[2000]	training's rmse: 0.539572	valid_1's rmse: 0.879684
[3000]	training's rmse: 0.530637	valid_1's rmse: 0.872033
[4000]	training's rmse: 0.523242	valid_1's rmse: 0.868843
[5000]	training's rmse: 0.516793	valid_1's rmse: 0.866499
[6000]	training's rmse: 0.511133	valid_1's rmse: 0.864411
[7000]	training's rmse: 0.506171	valid_1's rmse: 0.862744
[8000]	training's rmse: 0.501826	valid_1's rmse: 0.861463
[9000]	training's rmse: 0.498026	valid_1's rmse: 0.860475
[10000]	training's rmse: 0.494705	valid_1's rmse: 0.859752
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.494705	valid_1's rmse: 0.859752
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[1]	training's rmse: 0.736121	valid_1's rmse: 0.598927
Training until validation scores don't improve for 500 rounds.
[1000]	training's rmse: 0.720071	

In [150]:
mean_squared_error(train['score'], oof_preds)**0.5

0.6748730137759573

In [151]:
mean_squared_error(test['score'], sub)**0.5

0.6599172587499322

In [152]:
feature_importances.groupby('feature')[['importance']].mean()

,importance
feature,
eye_left_x,656.333333
eye_left_y,437.666667
eye_right_x,540.000000
eye_right_y,1174.000000
mouse_x,266.666667
mouse_y,683.333333
nose_x,132.333333
nose_y,96.000000


In [153]:
oof_preds

array([1.7672465 , 1.91744134, 2.11202973, 1.958625  , 1.88608491,
       1.9580947 , 1.958625  , 1.9580947 , 1.958625  , 1.9580947 ,
       1.60966203, 1.74143052, 1.8896342 , 1.958625  , 1.9383889 ,
       1.87770853, 1.958625  , 1.9580947 , 1.77945451, 1.9580947 ,
       1.71758104, 1.88196337, 2.324648  , 1.71332621, 1.9580947 ,
       1.9580947 , 2.13014765, 1.9580947 , 1.83267641, 1.71758104,
       1.68749324, 2.06742787, 1.71758104, 2.05425984, 1.9580947 ,
       1.78478175, 1.958625  , 1.91243767, 1.9580947 , 1.70854594,
       1.6314414 , 2.05134936, 1.958625  , 1.958625  , 1.72264319,
       1.958625  , 1.91744134, 1.958625  , 1.74805535, 1.68843472,
       1.9580947 , 2.07655176, 1.71758104, 1.958625  , 1.9580947 ,
       1.849339  , 2.11010121, 2.08155542, 1.9383889 , 1.94264374,
       1.91216943, 1.91744134, 1.7376886 , 1.74380051, 1.9580947 ,
       1.46167842, 1.81694758, 1.50397311, 1.958625  , 1.87016898,
       1.91258677, 2.07229692, 1.74380051, 1.98353096, 2.05134

In [156]:
test.drop(columns='score').to_csv('../new_face/test.csv', index=None)

In [154]:
sub

array([1.97098369, 2.08355731, 1.93605132, 1.91974619, 1.88408603,
       2.02423956, 1.77098051, 2.05523948, 1.99371866, 1.81448418])

In [155]:
train_test.score.var()

0.4815404040404041